# Segmentation Test

In [1]:
#where are the files to be processed?
source_folder="annotated_data"

#where are the files to be saved?
target_folder="eval_dataset"


In [6]:
#load libraries
import pandas as pd
#import pdf_segmenter_utils as kpdf
#import kutils as krusty
#import semantic_annotations_utils as sau
import numpy as np

In [5]:
paragraphs_df=pd.read_csv(source_folder+"/paragraphs_df_2023-12-03.csv")
styles_df=pd.read_csv(source_folder+"/styles_df_2023-12-03.csv")
documents_df=pd.read_csv(source_folder+"/documents_df_2023-12-03.csv")

In [6]:
#select a random sample of 40 documents from documents_df
sample_documents_df=documents_df.sample(n=40,random_state=1213)
sample_documents_df=sample_documents_df.sort_values(by=['id'])
#get paragrahs corresponding to the sample and sort by document_id and paragraph_id
sample_paragraphs_df=paragraphs_df[paragraphs_df.document.isin(sample_documents_df.id)].sort_values(by=['document','paragraph_id'])
#get styles corresponding to the sample and sort by document_id, paragraph_id, and paragraph_pos
sample_styles_df=styles_df[styles_df.document.isin(sample_documents_df.id)].sort_values(by=['document','paragraph_id','paragraph_pos'])


In [7]:
#save to csv
sample_documents_df.to_csv(target_folder+"/sample_documents_df_2023-11-30.csv",index=False)
sample_paragraphs_df.to_csv(target_folder+"/sample_paragraphs_df_2023-11-30.csv",index=False)
sample_styles_df.to_csv(target_folder+"/sample_styles_df_2023-11-30.csv",index=False)

In [8]:
#document statistics for all documents
print("Number of documents: "+str(len(documents_df)))

print("Non-segmented documents: ", documents_df[documents_df["num_sections"]==0].shape[0])

Number of documents: 1000
Non-segmented documents:  33


In [168]:
import pandas as pd
data=pd.read_csv('../../csv_files/Archive/df_semantic.csv')

In [169]:
data.head()

,id,document,paragraph_id,text_type,text_type_es,length,document_pos,paragraph_pos,text,section
0,0,0,0,emphasis,énfasis,22,0,0,Sentencia No. C-128/95,header
1,1,0,1,emphasis,énfasis,12,24,0,COSA JUZGADA,header
2,2,0,2,emphasis,énfasis,4,38,0,Ref:,header
3,3,0,2,plain,normal,18,42,4,Expediente D- 726,header
4,4,0,3,plain,normal,203,62,0,Demanda de inconstitucionalidad de los artícul...,header


In [170]:
sources_df=pd.read_csv('../../csv_files/Archive/df_docs.csv')

In [171]:
a=sources_df['name'].value_counts()
bad_indexes=[]
for i,j in zip(a.index,a.values):
    if j>1:
        bad_indexes.append(i)

In [172]:
save_dict={}
for idx in bad_indexes:
    dummy=sources_df[sources_df['name']==idx]
    save_dict[dummy['id'].values[1]]=dummy['id'].values[0]
    

In [173]:
def fix_duplicates(x,save_dict):
    try:
        return save_dict[x]
    except:
        return x

In [174]:
data['document']=data['document'].apply(lambda x: fix_duplicates(x,save_dict))

In [175]:
sources_df[sources_df['name']==bad_indexes[1]]

,id,name,type,year,num_sections,num_paragraphs
972,972,T-115-21,T,2021,3,164
1979,1979,T-115-21,T,2021,3,164


In [176]:
sources_df=sources_df.drop_duplicates(subset=['name','type','num_sections','num_paragraphs'])

In [177]:
sources_df=sources_df.rename(columns={'id':'docid'})

In [183]:
data=data.merge(sources_df[['name','docid']],how='left',left_on='document',right_on='docid')

In [178]:
#data.to_csv('/home/proyectocorte/csv_files/Archive/df_semantic_fixed.csv',index=False)

In [179]:
#sources_df.to_csv('/home/proyectocorte/csv_files/Archive/df_docs_fixed.csv',index=False)

In [185]:
import numpy as np
data['real_label']=np.nan

In [186]:
data=data[data['text_type_es']!='número de página']

In [187]:
sample_size=int(data.shape[0]*0.1)
sample_size=800

In [188]:
# inlcuir lo de los parrafos a futuro
data=data[['id','name','text','text_type_es','real_label']].reset_index().sample(sample_size)
data=data.rename(columns={'id':'old_id','index':'id'})
data['id']=[i for i in range(data.shape[0])]
data

,id,old_id,name,text,text_type_es,real_label
94678,0,100549,T-497-06,la sentencia proferida por el Juzgado Tercero ...,normal,NaN
252906,1,268479,C-466-08,Adopción como única posibilidad afecta la libe...,normal,NaN
251609,2,267083,T-577-19,.,énfasis_cita,NaN
522407,3,553670,T-102-17,"En el caso bajo estudio, se advierte que se cu...",normal,NaN
145353,4,154410,T-033-10,"Cfr. Corte Constitucional, Sentencias T-001 d...",pie de página,NaN
...,...,...,...,...,...,...
217381,795,231112,SU813-07,como configuratoria de una vía de hecho y se ...,normal,NaN
105156,796,111617,C-506-14,“(…) manejo de asuntos pertenecientes al exclu...,cita,NaN
665363,797,706007,C-108-00,"(Sent. C-222 de 1995, M.P. José Gregorio Herná...",normal,NaN
466544,798,494706,T-1164-04,procedencia de la tutela para lograr la garant...,cita,NaN


In [189]:
data=data.rename(columns={'text_type_es':'text_type'})

In [190]:
labels=list(data['text_type'].unique())
labels

['normal',
 'énfasis_cita',
 'pie de página',
 'cita',
 'número de pie de página',
 'pie de página_cita',
 'énfasis',
 'título',
 'título_cita',
 'pie de página_énfasis',
 'título_énfasis']

In [191]:
data

,id,old_id,name,text,text_type,real_label
94678,0,100549,T-497-06,la sentencia proferida por el Juzgado Tercero ...,normal,NaN
252906,1,268479,C-466-08,Adopción como única posibilidad afecta la libe...,normal,NaN
251609,2,267083,T-577-19,.,énfasis_cita,NaN
522407,3,553670,T-102-17,"En el caso bajo estudio, se advierte que se cu...",normal,NaN
145353,4,154410,T-033-10,"Cfr. Corte Constitucional, Sentencias T-001 d...",pie de página,NaN
...,...,...,...,...,...,...
217381,795,231112,SU813-07,como configuratoria de una vía de hecho y se ...,normal,NaN
105156,796,111617,C-506-14,“(…) manejo de asuntos pertenecientes al exclu...,cita,NaN
665363,797,706007,C-108-00,"(Sent. C-222 de 1995, M.P. José Gregorio Herná...",normal,NaN
466544,798,494706,T-1164-04,procedencia de la tutela para lograr la garant...,cita,NaN


In [192]:
import sqlite3
conn=sqlite3.connect('/home/proyectocorte/nlp_tagger/karolabeler/main_database_semantic.db')

In [193]:
data.to_sql(name='documents',con=conn,index=False)

800

In [194]:
data.head()

,id,old_id,name,text,text_type,real_label
94678,0,100549,T-497-06,la sentencia proferida por el Juzgado Tercero ...,normal,NaN
252906,1,268479,C-466-08,Adopción como única posibilidad afecta la libe...,normal,NaN
251609,2,267083,T-577-19,.,énfasis_cita,NaN
522407,3,553670,T-102-17,"En el caso bajo estudio, se advierte que se cu...",normal,NaN
145353,4,154410,T-033-10,"Cfr. Corte Constitucional, Sentencias T-001 d...",pie de página,NaN


In [29]:
import pandas as pd
security_df=pd.DataFrame({'user':['karolina.naranjov@gmail.com'],'password':['S3creT24*07']})
security_df

,user,password
0,karolina.naranjov@gmail.com,S3creT24*07


In [30]:
security_df.to_csv('user_psw.csv',index=False)

# fixing paragraph id

In [1]:
import pandas as pd
import sqlite3

In [ ]:

data=pd.read_csv('../../csv_files/Archive/df_semantic_fixed.csv')
sources_df=pd.read_csv('../../csv_files/Archive/df_docs_fixed.csv')
data=data.merge(sources_df[['name','docid']],how='left',left_on='document',right_on='docid')
conn=sqlite3.connect('/home/proyectocorte/nlp_tagger/karolabeler/main_database_semantic.db')

In [22]:
data_db=pd.read_sql("SELECT * FROM documents",conn)

In [23]:
data_db

,id,old_id,name,text,text_type,real_label
0,0,100549,T-497-06,la sentencia proferida por el Juzgado Tercero ...,normal,normal
1,1,268479,C-466-08,Adopción como única posibilidad afecta la libe...,normal,normal
2,2,267083,T-577-19,.,énfasis_cita,normal
3,3,553670,T-102-17,"En el caso bajo estudio, se advierte que se cu...",normal,normal
4,4,154410,T-033-10,"Cfr. Corte Constitucional, Sentencias T-001 d...",pie de página,pie de página
...,...,...,...,...,...,...
795,795,231112,SU813-07,como configuratoria de una vía de hecho y se ...,normal,None
796,796,111617,C-506-14,“(…) manejo de asuntos pertenecientes al exclu...,cita,None
797,797,706007,C-108-00,"(Sent. C-222 de 1995, M.P. José Gregorio Herná...",normal,None
798,798,494706,T-1164-04,procedencia de la tutela para lograr la garant...,cita,None


In [30]:
data[data['id']==128168]

,id,document,paragraph_id,text_type,text_type_es,length,document_pos,paragraph_pos,text,section,name,docid
128168,128168,285,34,quote,cita,680,6164,0,"El principio de legalidad de la sanción, como ...",header,C-211-00,285


In [25]:
final_data=data_db.merge(data[['id','paragraph_id','paragraph_pos']],how='left',left_on='old_id',right_on='id')

In [26]:
del final_data['id_y']
final_data=final_data.rename(columns={'id_x':'id'})
final_data

,id,old_id,name,text,text_type,real_label,paragraph_id,paragraph_pos
0,0,100549,T-497-06,la sentencia proferida por el Juzgado Tercero ...,normal,normal,100,17
1,1,268479,C-466-08,Adopción como única posibilidad afecta la libe...,normal,normal,56,175
2,2,267083,T-577-19,.,énfasis_cita,normal,204,1263
3,3,553670,T-102-17,"En el caso bajo estudio, se advierte que se cu...",normal,normal,123,4
4,4,154410,T-033-10,"Cfr. Corte Constitucional, Sentencias T-001 d...",pie de página,pie de página,106,1
...,...,...,...,...,...,...,...,...
795,795,231112,SU813-07,como configuratoria de una vía de hecho y se ...,normal,None,1084,769
796,796,111617,C-506-14,“(…) manejo de asuntos pertenecientes al exclu...,cita,None,108,759
797,797,706007,C-108-00,"(Sent. C-222 de 1995, M.P. José Gregorio Herná...",normal,None,118,717
798,798,494706,T-1164-04,procedencia de la tutela para lograr la garant...,cita,None,43,753


In [27]:
conn2=sqlite3.connect('/home/proyectocorte/nlp_tagger/karolabeler/main_database_semantic_1.db')
final_data.to_sql(name='documents',con=conn2,index=False)

800

In [29]:
final_data[final_data['id']==155]

,id,old_id,name,text,text_type,real_label,paragraph_id,paragraph_pos
155,155,128168,C-211-00,"El principio de legalidad de la sanción, como ...",cita,None,34,0


In [2]:
data=pd.read_csv('wetransfer_dataset-files_2024-12-09_1229/df_docs_fixed.csv')

In [3]:
data

,docid,name,type,year,num_sections,num_paragraphs
0,0,C-128-95,C,1995,3,87
1,1,C-843-10,C,2010,3,658
2,2,C-254A-12,C,2012,5,379
3,3,T-549-13,T,2013,4,123
4,4,T-011-99,T,1999,2,48
...,...,...,...,...,...,...
1990,1995,C-245-95,C,1995,9,185
1991,1996,C-287-00,C,2000,5,138
1992,1997,C-184-02,C,2002,0,165
1993,1998,T-074-99,T,1999,2,35


In [5]:
old_conn=sqlite3.connect('main_database_semantic_1.db')

In [33]:
old_db=pd.read_sql("SELECT * FROM documents",old_conn)

In [34]:
old_index=old_db['old_id'].values

In [50]:
big_df=pd.read_csv('wetransfer_dataset-files_2024-12-09_1229/df_semantic_fixed.csv')

In [51]:
big_df

,id,document,paragraph_id,text_type,text_type_es,length,document_pos,paragraph_pos,text,section
0,0,0,0,emphasis,énfasis,22,0,0,Sentencia No. C-128/95,header
1,1,0,1,emphasis,énfasis,12,24,0,COSA JUZGADA,header
2,2,0,2,emphasis,énfasis,4,38,0,Ref:,header
3,3,0,2,plain,normal,18,42,4,Expediente D- 726,header
4,4,0,3,plain,normal,203,62,0,Demanda de inconstitucionalidad de los artícul...,header
...,...,...,...,...,...,...,...,...,...,...
845642,845642,1999,202,plain,normal,52,45853,0,MARTHA VICTORIA SACHICA DE MONCALEANO Magistra...,Signature Block
845643,845643,1999,203,plain,normal,33,45907,0,VLADIMIRO NARANJO MESA Magistrado,Signature Block
845644,845644,1999,204,page_number,número de página,13,45942,0,Página No. 21,Signature Block
845645,845645,1999,205,plain,normal,46,45956,0,PLABO ENRIQUE LEAL RUIZ Secretario General (E),Signature Block


In [60]:
new_sample=big_df[~big_df['id'].isin(old_index)].sample(800)

In [61]:
new_sample=new_sample[['id','document','text','paragraph_id','paragraph_pos','text_type_es']]

In [62]:
new_sample

,id,document,text,paragraph_id,paragraph_pos,text_type_es
431510,431510,817,"Que según la misma norma constitucional, una v...",54,0,normal
778558,778558,1779,"Ver entre otras las Sentencias C-252 de 2001,...",189,143,pie de página
716533,716533,1570,"""Mediante la comercialización en el país de lo...",158,0,normal
372854,372854,729,habitantes y solucionar el acceso a la educaci...,299,282,normal
429139,429139,816,130,144,189,número de pie de página
...,...,...,...,...,...,...
382230,382230,741,Página No. 123,4282,0,número de página
686173,686173,1462,"Finalmente, encuentra la Sala pertinente resal...",111,0,normal
553420,553420,1036,INHIBIDA,142,11,énfasis
14173,14173,24,33,622,338,número de pie de página


In [63]:
new_sample=new_sample.merge(data[['name','docid']],how='left',left_on='document',right_on='docid')

In [64]:
new_sample=new_sample.rename(columns={'text_type_es':'text_type','id':'old_id'})
new_sample

,old_id,document,text,paragraph_id,paragraph_pos,text_type,name,docid
0,431510,817,"Que según la misma norma constitucional, una v...",54,0,normal,C-223-11,817
1,778558,1779,"Ver entre otras las Sentencias C-252 de 2001,...",189,143,pie de página,C-225-19,1779
2,716533,1570,"""Mediante la comercialización en el país de lo...",158,0,normal,C-109-96,1570
3,372854,729,habitantes y solucionar el acceso a la educaci...,299,282,normal,T-329-10,729
4,429139,816,130,144,189,número de pie de página,T-430-21,816
...,...,...,...,...,...,...,...,...
795,382230,741,Página No. 123,4282,0,número de página,T-447-18,741
796,686173,1462,"Finalmente, encuentra la Sala pertinente resal...",111,0,normal,T-077-05,1462
797,553420,1036,INHIBIDA,142,11,énfasis,C-251-04,1036
798,14173,24,33,622,338,número de pie de página,C-1300-05,24


In [65]:
new_sample=new_sample.reset_index()

In [66]:
new_sample= new_sample.rename(columns={'index':'id'})
new_sample

,id,old_id,document,text,paragraph_id,paragraph_pos,text_type,name,docid
0,0,431510,817,"Que según la misma norma constitucional, una v...",54,0,normal,C-223-11,817
1,1,778558,1779,"Ver entre otras las Sentencias C-252 de 2001,...",189,143,pie de página,C-225-19,1779
2,2,716533,1570,"""Mediante la comercialización en el país de lo...",158,0,normal,C-109-96,1570
3,3,372854,729,habitantes y solucionar el acceso a la educaci...,299,282,normal,T-329-10,729
4,4,429139,816,130,144,189,número de pie de página,T-430-21,816
...,...,...,...,...,...,...,...,...,...
795,795,382230,741,Página No. 123,4282,0,número de página,T-447-18,741
796,796,686173,1462,"Finalmente, encuentra la Sala pertinente resal...",111,0,normal,T-077-05,1462
797,797,553420,1036,INHIBIDA,142,11,énfasis,C-251-04,1036
798,798,14173,24,33,622,338,número de pie de página,C-1300-05,24


In [67]:
del new_sample['docid']
del new_sample['document']

In [70]:
new_sample=new_sample[['id','old_id','name','text','text_type','paragraph_id','paragraph_pos']]

In [72]:
import numpy as np
new_sample['real_label']=np.nan

/tmp/ipykernel_16809/226026198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sample['real_label']=np.nan


In [76]:
new_sample=new_sample[['id','old_id','name','text','text_type','real_label','paragraph_id','paragraph_pos']]

In [77]:
conn2=sqlite3.connect('main_database_semantic_2.db')
new_sample.to_sql(name='documents',con=conn2,index=False)

800